In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

from pyspark.sql.utils import AnalysisException

In [3]:
spark=SparkSession.builder.appName('Comprehensive_assesment2').getOrCreate()


22/05/14 21:57:30 WARN Utils: Your hostname, sonus-MacBook.local resolves to a loopback address: 127.0.0.1; using 192.168.0.101 instead (on interface en0)
22/05/14 21:57:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/14 21:57:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/14 21:57:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/14 21:57:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
spark

In [7]:
df=spark.read.option("header",'true').csv('covid.csv',inferSchema=True)

In [8]:
df.columns

['Date.Day',
 'Date.Month',
 'Date.Year',
 'Data.Cases',
 'Data.Deaths',
 'Location.Country',
 'Location.Code',
 'Data.Population',
 'Location.Continent',
 'Data.Rate']

In [9]:
n_df=df.withColumnRenamed('Date.Day','Day').withColumnRenamed('Date.Month','Month').withColumnRenamed('Date.Year','Year').withColumnRenamed('Data.Cases','Cases').withColumnRenamed('Data.Deaths','Deaths').withColumnRenamed('Location.Country','Country').withColumnRenamed('Location.Code','Code').withColumnRenamed('Data.Population','Population').withColumnRenamed('Location.Continent','Continent').withColumnRenamed('Data.Rate','Rate')










In [10]:
n_df.columns

['Day',
 'Month',
 'Year',
 'Cases',
 'Deaths',
 'Country',
 'Code',
 'Population',
 'Continent',
 'Rate']

In [11]:
#1. Find out the total number of Countries which has Cases greater than 500.
n_df.filter(n_df['Cases']>500).agg({'Country':'count'}).show()

+--------------+
|count(Country)|
+--------------+
|          9077|
+--------------+



In [14]:
#2. Find the Codes which has negative Deaths (Ex: -9,-5).
n_df.filter(n_df['Deaths']<0).select("Code").show()

+----+
|Code|
+----+
| ESP|
| ITA|
| CZE|
| CZE|
| XKX|
| ESP|
| KGZ|
| IRL|
+----+



In [73]:
#3. Which date has the highest Rate?
df1=n_df.groupBy(['Day','Month','Year']).agg({'Rate':'max'})
df1.orderBy(df1['max(Rate)'].desc()).show(1)

+---+-----+----+----------+
|Day|Month|Year| max(Rate)|
+---+-----+----+----------+
|  2|   11|2020|1900.83621|
+---+-----+----+----------+
only showing top 1 row



In [46]:
#4. Which month has minimum Death?
df1=n_df.groupBy('Month').agg({'Deaths':'min'})
df1.orderBy(df1['min(Deaths)'].desc()).show(1)

+-----+-----------+
|Month|min(Deaths)|
+-----+-----------+
|   12|          0|
+-----+-----------+
only showing top 1 row



In [78]:
#5. Which cities have Populations lesser than 50,000?
n_df.filter(n_df['Population']<500000).select('Country').show()

+--------------------+
|             Country|
+--------------------+
|Cases_on_an_inter...|
|             Iceland|
|              Monaco|
|          San_Marino|
|Cases_on_an_inter...|
|             Iceland|
|              Monaco|
|          San_Marino|
|Cases_on_an_inter...|
|             Iceland|
|              Monaco|
|          San_Marino|
|Cases_on_an_inter...|
|             Iceland|
|              Monaco|
|          San_Marino|
|Cases_on_an_inter...|
|             Iceland|
|              Monaco|
|          San_Marino|
+--------------------+
only showing top 20 rows



In [49]:
#6. Find Dates with Continent =’ Oceania’ and have Deaths greater than 20 and less than 50
n_df.filter((n_df['Deaths']>20) & (n_df['Deaths']>50) &(n_df['Continent']=='Oceania')).select(['Day','Month','Year']).show()

+---+-----+----+
|Day|Month|Year|
+---+-----+----+
|  5|    9|2020|
+---+-----+----+



In [50]:
#7. Find the Countries, Cases, Deaths, Rate for all the Countries in the Continent =’ Asia’.

n_df.filter(n_df['Continent']=='Asia').select(['Country','Cases','Deaths','Rate']).show()

+-----------+-----+------+----+
|    Country|Cases|Deaths|Rate|
+-----------+-----+------+----+
|Afghanistan|    0|     0| 0.0|
|    Bahrain|    0|     0| 0.0|
|   Cambodia|    0|     0| 0.0|
|      China|   27|     0| 0.0|
|      India|    0|     0| 0.0|
|  Indonesia|    0|     0| 0.0|
|       Iran|    0|     0| 0.0|
|       Iraq|    0|     0| 0.0|
|     Israel|    0|     0| 0.0|
|      Japan|    0|     0| 0.0|
|     Kuwait|    0|     0| 0.0|
|    Lebanon|    0|     0| 0.0|
|   Malaysia|    0|     0| 0.0|
|      Nepal|    0|     0| 0.0|
|       Oman|    0|     0| 0.0|
|   Pakistan|    0|     0| 0.0|
|Philippines|    0|     0| 0.0|
|      Qatar|    0|     0| 0.0|
|  Singapore|    0|     0| 0.0|
|South_Korea|    0|     0| 0.0|
+-----------+-----+------+----+
only showing top 20 rows



In [70]:
#8. On ‘24/5/2020’ which Country, Continent has Least Population and Highest Population?
df_5=n_df.filter((n_df['Day']=='25')& (n_df['Month']=='5')&(n_df['Year']=='2020')).groupBy(['Continent','Country']).agg({'Population':'max'})
df_5.orderBy(df_5['max(Population)'].desc()).show(1)




+---------+-------+---------------+
|Continent|Country|max(Population)|
+---------+-------+---------------+
|     Asia|  China|     1433783692|
+---------+-------+---------------+
only showing top 1 row



In [72]:
df_6=n_df.filter((n_df['Day']=='25')& (n_df['Month']=='5')&(n_df['Year']=='2020')).groupBy(['Continent','Country']).agg({'Population':'min'})
df_6.orderBy(df_6['min(Population)'].asc()).show(1)

+---------+--------+---------------+
|Continent| Country|min(Population)|
+---------+--------+---------------+
|   Europe|Holy_See|            815|
+---------+--------+---------------+
only showing top 1 row



In [74]:
print(f"I am sonu Mahto")

I am sonu Mahto
